In [5]:
import os
import sys
sys.path.append('../../') 

from main import FileParser, run_arg
def import_config(folderpaths:str="heat"):
    abspath = "/cluster/work/math/shiwen/GNPDESolver"
    folderpaths = os.path.join(abspath, folderpaths)
    filenames = os.listdir(folderpaths)
    configpaths = []
    for filename in filenames:
        if filename.endswith(".json"):
            configpaths.append(os.path.join(folderpaths, filename))
    return configpaths

def as_abs_path(_path):
    if not os.path.isabs(_path):
        _path = os.path.join(
            os.path.abspath(os.path.dirname(os.path.dirname(os.getcwd()))),
            _path
            )

    return _path

In [6]:
foldername = "config/gaot/fluid_data/sparse/geoemb/stats"
filenames = import_config(foldername)
print(f"num configs:{len(filenames)}")
for filename in filenames:
    print(filename)

num configs:1
/cluster/work/math/shiwen/GNPDESolver/config/gaot/fluid_data/sparse/geoemb/stats/ce_rp.json


In [12]:
args = FileParser(filenames[0]).parse_args()
print(args.setup)


{'seed': 42, 'device': 'cuda:0', 'dtype': 'torch.float32', 'trainer_name': 'sequential', 'train': True, 'test': False, 'ckpt': False, 'use_variance_test': False}


In [ ]:
# ==== 参考对方给出的配色和一些全局配置 ====
C_BLACK = '#000000'
C_WHITE = '#ffffff'
C_BLUE = '#093691'
C_RED = '#911b09'
C_BLACK_BLUEISH = '#011745'
C_BLACK_REDDISH = '#380801'
C_WHITE_BLUEISH = '#dce5f5'
C_WHITE_REDDISH = '#f5dcdc'

CMAP_BBR = matplotlib.colors.LinearSegmentedColormap.from_list(
    'blue_black_red',
    [C_WHITE_BLUEISH, C_BLUE, C_BLACK, C_RED, C_WHITE_REDDISH],
    N=200,
)
CMAP_BWR = matplotlib.colors.LinearSegmentedColormap.from_list(
    'blue_white_red',
    [C_BLACK_BLUEISH, C_BLUE, C_WHITE, C_RED, C_BLACK_REDDISH],
    N=200,
)
CMAP_WRB = matplotlib.colors.LinearSegmentedColormap.from_list(
    'white_red_black',
    [C_WHITE, C_RED, C_BLACK],
    N=200,
)

# 全局字体
plt.rcParams['font.family'] = 'serif'

    def plot_results(self, coords, input, gt_sequence, pred_sequence, time_indices, t_values, num_frames=5):
        """
        用 scatter 绘图：
        - 只绘制第一帧、最后一帧
        - 行方向: 每个变量3行 (GT, Pred, Error)
        - 列方向: 2列 (第一帧, 最后一帧)
        """

        num_timesteps = gt_sequence.shape[0]
        num_nodes = coords.shape[0]
        num_vars = gt_sequence.shape[-1]

        # 只取第一帧和最后一帧
        frame_indices = [0, num_timesteps - 1]

        x = coords[:, 0]
        y = coords[:, 1]

        # 先算全局最小最大值，以便GT和Pred用统一的色标
        vmin_list = []
        vmax_list = []
        for var_idx in range(num_vars):
            all_gt = gt_sequence[..., var_idx]
            all_pred = pred_sequence[..., var_idx]
            min_val = min(all_gt.min(), all_pred.min())
            max_val = max(all_gt.max(), all_pred.max())
            vmin_list.append(min_val)
            vmax_list.append(max_val)

        # figure 大小，你可以根据需求调大或调小
        fig_height_per_var = 2.2
        fig_width_per_frame = 3.0
        fig = plt.figure(figsize=(fig_width_per_frame * 2, fig_height_per_var * 3 * num_vars))

        # 3*num_vars行, 2列
        axes = []
        for i in range(3 * num_vars):
            row_axes = []
            for j in range(2):
                ax = plt.subplot(3 * num_vars, 2, i * 2 + j + 1)
                row_axes.append(ax)
            axes.append(row_axes)

        # 散点设置，可根据需要调参
        scatter_settings = dict(
            s=10,           # 点大小
            alpha=1.0,
            marker='o',
            linewidth=0.0,
        )

        for var_idx in range(num_vars):
            vmin = vmin_list[var_idx]
            vmax = vmax_list[var_idx]

            for icol, frame_idx in enumerate(frame_indices):
                # 对应的物理时间
                time_idx = time_indices[frame_idx]
                time_val = t_values[time_idx]

                gt = gt_sequence[frame_idx, :, var_idx]
                pred = pred_sequence[frame_idx, :, var_idx]
                abs_error = np.abs(gt - pred)

                row_gt = var_idx * 3
                row_pred = var_idx * 3 + 1
                row_err = var_idx * 3 + 2

                # ===== Ground Truth =====
                ax_gt = axes[row_gt][icol]
                sc_gt = ax_gt.scatter(
                    x, y,
                    c=gt,
                    cmap=CMAP_BWR,
                    vmin=vmin,
                    vmax=vmax,
                    **scatter_settings
                )
                ax_gt.set_aspect('equal')
                ax_gt.set_xticks([])
                ax_gt.set_yticks([])
                cb_gt = plt.colorbar(sc_gt, ax=ax_gt, orientation='vertical', fraction=0.046, pad=0.04)
                if icol == 0:
                    ax_gt.set_ylabel(f'Var {var_idx + 1} | GT', fontsize=10)
                if var_idx == 0:
                    ax_gt.set_title(f"Time: {time_val:.2f}", fontsize=10)

                # ===== Prediction =====
                ax_pred = axes[row_pred][icol]
                sc_pred = ax_pred.scatter(
                    x, y,
                    c=pred,
                    cmap=CMAP_BWR,
                    vmin=vmin,
                    vmax=vmax,
                    **scatter_settings
                )
                ax_pred.set_aspect('equal')
                ax_pred.set_xticks([])
                ax_pred.set_yticks([])
                cb_pred = plt.colorbar(sc_pred, ax=ax_pred, orientation='vertical', fraction=0.046, pad=0.04)
                if icol == 0:
                    ax_pred.set_ylabel('Pred', fontsize=10)

                # ===== Error =====
                ax_err = axes[row_err][icol]
                # 可以换一个更适合 [0, max_err] 的颜色表，比如 CMAP_WRB
                # 注意 vmin=0
                max_err = abs_error.max()
                sc_err = ax_err.scatter(
                    x, y,
                    c=abs_error,
                    cmap=CMAP_WRB,
                    vmin=0,
                    vmax=max_err,
                    **scatter_settings
                )
                ax_err.set_aspect('equal')
                ax_err.set_xticks([])
                ax_err.set_yticks([])
                cb_err = plt.colorbar(sc_err, ax=ax_err, orientation='vertical', fraction=0.046, pad=0.04)
                if icol == 0:
                    ax_err.set_ylabel('|Error|', fontsize=10)

        plt.tight_layout()
        plt.savefig(self.path_config.result_path, dpi=300)
        plt.close()
        
        def measure_inference_time(self):
            """
            Measure the inference time of the model and print the average over multiple runs.

            Returns:
                None
            """

            import time
            self.model.eval()
            self.model.to(self.device)
            with torch.no_grad():
                # Get a single sample from the test dataset
                x_sample, y_sample = self.test_dataset[0]
                # Ensure inputs are tensors and add batch dimension
                x_sample = torch.tensor(x_sample, dtype=self.dtype).unsqueeze(0).to(self.device)  # Shape: [1, num_nodes, input_dim]
                # Warm-up run
                _ = self.model(self.rigraph, x_sample)
                # Measure inference time over 10 runs
                times = []
                for _ in range(10):
                    start_time = time.perf_counter()
                    if self.model_config.use_conditional_norm:
                        pred = self.model(self.rigraph, x_sample[...,:-1], x_sample[...,0,-2:-1])
                    else:
                        pred = self.model(self.rigraph, x_sample)
                    # Ensure all CUDA kernels have finished before stopping the timer
                    if 'cuda' in str(self.device):
                        torch.cuda.synchronize()
                    end_time = time.perf_counter()
                    times.append(end_time - start_time)
                avg_time = sum(times) / len(times)
                print(f"Average inference time over 10 runs (batch size = 1): {avg_time:.6f} seconds")
